In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-10 13:48:00.999266: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 13:48:01.099885: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 13:48:01.099902: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 13:48:01.120658: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 13:48:01.701898: W tensorflow/stream_executor/platform/de

In [2]:
name_all=[]
auc_roc=[]
name='all_algorithm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

2022-11-10 13:48:07.386918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 13:48:07.388491: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:48:07.388643: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:48:07.388777: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-10 13:48:07.388905: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

Epoch 1/100
1/1 [==============================] - 0s 452ms/step - loss: 2.3520 - auc: 0.7309
Epoch 2/100
1/1 [==============================] - 0s 33ms/step - loss: 2.3724 - auc: 0.8454
Epoch 3/100
1/1 [==============================] - 0s 40ms/step - loss: 2.0883 - auc: 0.8785
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.8408 - auc: 0.8994
Epoch 5/100
1/1 [==============================] - 0s 41ms/step - loss: 1.7053 - auc: 0.9134
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.6055 - auc: 0.9282
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4729 - auc: 0.9595
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3553 - auc: 0.9784
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2619 - auc: 0.9869
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1848 - auc: 0.9919
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 30ms/step - loss: 0.1448 - auc: 0.9976
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1455 - auc: 0.9973
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1439 - auc: 0.9984
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1459 - auc: 0.9977
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1415 - auc: 0.9982
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1490 - auc: 0.9970
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1468 - auc: 0.9975
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1443 - auc: 0.9981
Epoch 96/100
1/1 [==============================] - 0s 33ms/step - loss: 0.1412 - auc: 0.9982
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1420 - auc: 0.9981
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [3]:
name='remove_rf'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 347ms/step - loss: 2.1838 - auc: 0.6192
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 2.0477 - auc: 0.8297
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 2.0380 - auc: 0.8462
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8615 - auc: 0.8650
Epoch 5/100
1/1 [==============================] - 0s 24ms/step - loss: 1.6299 - auc: 0.8902
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4890 - auc: 0.9028
Epoch 7/100
1/1 [==============================] - 0s 23ms/step - loss: 1.4451 - auc: 0.8944
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 1.3822 - auc: 0.9034
Epoch 9/100
1/1 [==============================] - 0s 26ms/step - loss: 1.2827 - auc: 0.9321
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1600 - auc: 0.9643
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 30ms/step - loss: 0.1815 - auc: 0.9955
Epoch 89/100
1/1 [==============================] - 0s 44ms/step - loss: 0.1832 - auc: 0.9958
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1904 - auc: 0.9951
Epoch 91/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1865 - auc: 0.9951
Epoch 92/100
1/1 [==============================] - 0s 35ms/step - loss: 0.1824 - auc: 0.9960
Epoch 93/100
1/1 [==============================] - 0s 40ms/step - loss: 0.1884 - auc: 0.9949
Epoch 94/100
1/1 [==============================] - 0s 35ms/step - loss: 0.1812 - auc: 0.9962
Epoch 95/100
1/1 [==============================] - 0s 39ms/step - loss: 0.1860 - auc: 0.9953
Epoch 96/100
1/1 [==============================] - 0s 40ms/step - loss: 0.1815 - auc: 0.9957
Epoch 97/100
1/1 [==============================] - 0s 38ms/step - loss: 0.1843 - auc: 0.9954
Epoch 98/100
1/1 [==============================] - 0s 35ms/step - loss: 

In [4]:
name='remove_dmpnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 456ms/step - loss: 2.6054 - auc: 0.3936
Epoch 2/100
1/1 [==============================] - 0s 37ms/step - loss: 1.8931 - auc: 0.8224
Epoch 3/100
1/1 [==============================] - 0s 36ms/step - loss: 1.9400 - auc: 0.8735
Epoch 4/100
1/1 [==============================] - 0s 37ms/step - loss: 1.9978 - auc: 0.8840
Epoch 5/100
1/1 [==============================] - 0s 42ms/step - loss: 1.9441 - auc: 0.8927
Epoch 6/100
1/1 [==============================] - 0s 39ms/step - loss: 1.8150 - auc: 0.9096
Epoch 7/100
1/1 [==============================] - 0s 38ms/step - loss: 1.6269 - auc: 0.9223
Epoch 8/100
1/1 [==============================] - 0s 36ms/step - loss: 1.4731 - auc: 0.9337
Epoch 9/100
1/1 [==============================] - 0s 37ms/step - loss: 1.3144 - auc: 0.9518
Epoch 10/100
1/1 [==============================] - 0s 38ms/step - loss: 1.2311 - auc: 0.9622
Epoch 11/100
1/1 [==============================] - 0s 38ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1706 - auc: 0.9966
Epoch 89/100
1/1 [==============================] - 0s 34ms/step - loss: 0.1719 - auc: 0.9962
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1668 - auc: 0.9969
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1668 - auc: 0.9971
Epoch 92/100
1/1 [==============================] - 0s 36ms/step - loss: 0.1644 - auc: 0.9975
Epoch 93/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1622 - auc: 0.9976
Epoch 94/100
1/1 [==============================] - 0s 35ms/step - loss: 0.1641 - auc: 0.9971
Epoch 95/100
1/1 [==============================] - 0s 33ms/step - loss: 0.1685 - auc: 0.9962
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1621 - auc: 0.9976
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1636 - auc: 0.9973
Epoch 98/100
1/1 [==============================] - 0s 32ms/step - loss: 

In [5]:
name='remove_lr'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 385ms/step - loss: 2.5991 - auc: 0.3878
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.8553 - auc: 0.8248
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 1.9010 - auc: 0.8779
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.9637 - auc: 0.8913
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.9444 - auc: 0.8984
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.8237 - auc: 0.9094
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6537 - auc: 0.9203
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 1.4817 - auc: 0.9328
Epoch 9/100
1/1 [==============================] - 0s 37ms/step - loss: 1.3203 - auc: 0.9489
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2174 - auc: 0.9628
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.1777 - auc: 0.9964
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1752 - auc: 0.9967
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1773 - auc: 0.9960
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1737 - auc: 0.9968
Epoch 92/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1759 - auc: 0.9969
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1677 - auc: 0.9975
Epoch 94/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1727 - auc: 0.9967
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1770 - auc: 0.9957
Epoch 96/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1729 - auc: 0.9963
Epoch 97/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1695 - auc: 0.9967
Epoch 98/100
1/1 [==============================] - 0s 31ms/step - loss: 

In [6]:
name='remove_fcnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 394ms/step - loss: 2.2967 - auc: 0.5298
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8930 - auc: 0.8459
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 1.9386 - auc: 0.8804
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.8869 - auc: 0.8960
Epoch 5/100
1/1 [==============================] - 0s 26ms/step - loss: 1.7292 - auc: 0.9134
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5588 - auc: 0.9306
Epoch 7/100
1/1 [==============================] - 0s 35ms/step - loss: 1.3813 - auc: 0.9493
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2673 - auc: 0.9629
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2146 - auc: 0.9663
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1597 - auc: 0.9714
Epoch 11/100
1/1 [==============================] - 0s 35ms/step - l

1/1 [==============================] - 0s 36ms/step - loss: 0.1485 - auc: 0.9981
Epoch 89/100
1/1 [==============================] - 0s 35ms/step - loss: 0.1489 - auc: 0.9981
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1498 - auc: 0.9976
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1515 - auc: 0.9973
Epoch 92/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1459 - auc: 0.9977
Epoch 93/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1494 - auc: 0.9976
Epoch 94/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1518 - auc: 0.9971
Epoch 95/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1501 - auc: 0.9976
Epoch 96/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1444 - auc: 0.9979
Epoch 97/100
1/1 [==============================] - 0s 33ms/step - loss: 0.1424 - auc: 0.9985
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [7]:
name='remove_xgb'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['fcnn','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 348ms/step - loss: 2.1837 - auc: 0.7755
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.9406 - auc: 0.7574
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 1.7977 - auc: 0.7816
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6264 - auc: 0.8596
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4673 - auc: 0.9184
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3990 - auc: 0.9299
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2713 - auc: 0.9548
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1765 - auc: 0.9695
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1094 - auc: 0.9770
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 1.0411 - auc: 0.9845
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1743 - auc: 0.9963
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1802 - auc: 0.9954
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1708 - auc: 0.9965
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1679 - auc: 0.9972
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1735 - auc: 0.9962
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1706 - auc: 0.9965
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1729 - auc: 0.9965
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1746 - auc: 0.9960
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1646 - auc: 0.9977
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1695 - auc: 0.9970
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [8]:
de=pd.DataFrame()
de['name']=name_all
de['auc_roc']=auc_roc
de.to_csv('5model_remove_one_model.csv',index=False)
print(de)

            name   auc_roc
0  all_algorithm  0.940468
1      remove_rf  0.945067
2   remove_dmpnn  0.939571
3      remove_lr  0.934512
4    remove_fcnn  0.941203
5     remove_xgb  0.936707
